<a href="https://colab.research.google.com/github/ipeirotis/introduction-to-databases/blob/master/module4/D-SQL_Aggregation_Queries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL: Aggregation Queries

**Learning Outcomes:**

By the end of this lesson, you will be able to:

- Use aggregation functions (COUNT, SUM, AVG, MIN, MAX, STDDEV) to summarize data
- Understand the difference between COUNT(*), COUNT(column), and COUNT(DISTINCT column)
- Use GROUP BY to compute aggregates for subgroups of data
- Use GROUP BY with multiple attributes
- Use the HAVING clause to filter groups based on aggregate values
- Understand the difference between WHERE (filters rows) and HAVING (filters groups)
- Combine JOIN and GROUP BY queries to compute statistics across multiple tables

## Setup

In [ ]:
# Authenticate with Google Cloud
from google.colab import auth
auth.authenticate_user()

In [ ]:
from google.cloud import bigquery

# Specify your Google Cloud project ID
PROJECT_ID = 'nyu-datasets'  # <-- Replace with your project ID

client = bigquery.Client(project=PROJECT_ID)

def run_query(sql):
    """Run a BigQuery SQL query and return results as a pandas DataFrame."""
    return client.query(sql).to_dataframe()

---
## Basic Aggregation Functions

Aggregation functions compute a single result from a set of input values:

| Function | Description |
|----------|-------------|
| `COUNT(*)` | Count all rows |
| `COUNT(column)` | Count non-NULL values |
| `COUNT(DISTINCT column)` | Count unique values |
| `SUM(column)` | Sum of values |
| `AVG(column)` | Average of values |
| `MIN(column)` | Minimum value |
| `MAX(column)` | Maximum value |
| `STDDEV(column)` | Standard deviation |

### `COUNT(*)`

#### Find the number of movies in the database

In [ ]:
run_query("""
SELECT COUNT(*) AS num_movies
FROM `nyu-datasets.imdb.movies`
""")

#### Find the number of actors in the database

In [ ]:
run_query("""
SELECT COUNT(*) AS num_actors
FROM `nyu-datasets.imdb.actors`
""")

### `COUNT(column)`

`COUNT(column)` counts only non-NULL values in that column.

#### Find the number of movies with a rating

In [ ]:
run_query("""
SELECT COUNT(rating) AS rated_movies
FROM `nyu-datasets.imdb.movies`
""")

#### Find the number of roles where the role name is not empty

In [ ]:
run_query("""
SELECT COUNT(role) AS named_roles
FROM `nyu-datasets.imdb.roles`
""")

In [ ]:
# Equivalent using WHERE
run_query("""
SELECT COUNT(*) AS named_roles
FROM `nyu-datasets.imdb.roles`
WHERE role IS NOT NULL
""")

,named_roles
0,2511546


### `COUNT(DISTINCT column)`

`COUNT(DISTINCT column)` counts unique non-NULL values.

#### Find the number of distinct genres in the database

In [ ]:
run_query("""
SELECT COUNT(DISTINCT genre) AS num_genres
FROM `nyu-datasets.imdb.movies_genres`
""")

,num_genres
0,21


#### Find the number of movies that have a genre associated with them

In [ ]:
run_query("""
SELECT COUNT(DISTINCT movie_id) AS num_movies_with_genre
FROM `nyu-datasets.imdb.movies_genres`
""")

,num_movies_with_genre
0,251142


Compare with the (incorrect!) query below without `DISTINCT`. Without `DISTINCT`, we count the same `movie_id` multiple times if it has multiple genres:

In [ ]:
run_query("""
SELECT COUNT(movie_id) AS count_with_duplicates
FROM `nyu-datasets.imdb.movies_genres`
""")

,count_with_duplicates
0,395450


### `MIN`, `MAX`, `AVG`, `STDDEV`, `SUM`

#### Find the earliest and latest release year for movies

In [ ]:
run_query("""
SELECT
    MIN(year) AS earliest_year,
    MAX(year) AS latest_year
FROM `nyu-datasets.imdb.movies`
""")

,earliest_year,latest_year
0,1888,2008


#### Find the average rating of movies and the standard deviation

In [ ]:
run_query("""
SELECT
    MIN(rating) AS min_rating,
    MAX(rating) AS max_rating,
    ROUND(AVG(rating), 2) AS avg_rating,
    ROUND(STDDEV(rating), 2) AS stdev_rating
FROM `nyu-datasets.imdb.movies`
""")

,min_rating,max_rating,avg_rating,stdev_rating
0,1.000000000,9.900000000,5.870000000,1.62


---
## `GROUP BY`

GROUP BY splits rows into groups and computes aggregates for each group.

```sql
SELECT column1, column2, AggFunc(column3)
FROM table
GROUP BY column1, column2
```

**Important:** Every column in SELECT (except aggregates) must appear in GROUP BY.

### GROUP BY Examples: IMDB

#### Count the number of movies released in each year

In [ ]:
run_query("""
SELECT year, COUNT(*) AS num_movies
FROM `nyu-datasets.imdb.movies`
GROUP BY year
ORDER BY year
""")

,year,num_movies
0,1888,2
1,1890,3
2,1891,6
3,1892,9
4,1893,2
...,...,...
115,2004,8718
116,2005,1449
117,2006,195
118,2007,7


#### Compute the average rating for movies released in each year

In [ ]:
run_query("""
SELECT year, ROUND(AVG(rating), 2) AS avg_rating
FROM `nyu-datasets.imdb.movies`
GROUP BY year
ORDER BY year
""")

,year,avg_rating
0,1888,None
1,1890,7.300000000
2,1891,3.680000000
3,1892,2.870000000
4,1893,6.800000000
...,...,...
115,2004,6.220000000
116,2005,None
117,2006,None
118,2007,None


#### Compute min, max, and standard deviation of ratings per year

In [ ]:
run_query("""
SELECT
    year,
    MIN(rating) AS min_rating,
    MAX(rating) AS max_rating,
    ROUND(AVG(rating), 2) AS avg_rating,
    ROUND(STDDEV(rating), 2) AS stdev_rating
FROM `nyu-datasets.imdb.movies`
GROUP BY year
ORDER BY year
""")

#### Examine the difference between `COUNT(*)` and `COUNT(rating)`

In [ ]:
run_query("""
SELECT
    year,
    COUNT(*) AS num_movies,
    COUNT(rating) AS rated_movies,
    ROUND(AVG(rating), 2) AS avg_rating
FROM `nyu-datasets.imdb.movies`
GROUP BY year
ORDER BY year DESC
""")

#### Compute the number of movies per director ID (rank by most movies)

In [ ]:
run_query("""
SELECT director_id, COUNT(*) AS num_movies
FROM `nyu-datasets.imdb.movies_directors`
GROUP BY director_id
ORDER BY num_movies DESC
LIMIT 20
""")

#### Compute the number of movies per actor ID (rank by most movies)

In [ ]:
run_query("""
SELECT actor_id, COUNT(*) AS num_movies
FROM `nyu-datasets.imdb.roles`
GROUP BY actor_id
ORDER BY num_movies DESC
LIMIT 20
""")

#### Compute the number of actors per movie ID (rank by most actors)

In [ ]:
run_query("""
SELECT
    movie_id,
    COUNT(*) AS num_roles,
    COUNT(DISTINCT actor_id) AS num_actors
FROM `nyu-datasets.imdb.roles`
GROUP BY movie_id
ORDER BY num_actors DESC
LIMIT 20
""")

#### Count the number of male and female actors

In [ ]:
run_query("""
SELECT gender, COUNT(*) AS count
FROM `nyu-datasets.imdb.actors`
GROUP BY gender
""")

#### Compute the number of movies for each genre

In [ ]:
run_query("""
SELECT
    genre,
    COUNT(DISTINCT movie_id) AS num_movies,
    COUNT(movie_id) AS count_with_duplicates
FROM `nyu-datasets.imdb.movies_genres`
GROUP BY genre
ORDER BY num_movies DESC
""")

### GROUP BY Examples: Facebook

#### List the number of males and females

In [ ]:
run_query("""
SELECT Sex, COUNT(*) AS cnt
FROM `nyu-datasets.facebook.Profiles`
GROUP BY Sex
""")

#### List the number of students for each political view

In [ ]:
run_query("""
SELECT PoliticalViews, COUNT(*) AS cnt
FROM `nyu-datasets.facebook.Profiles`
GROUP BY PoliticalViews
ORDER BY cnt DESC
""")

#### List the number of males and female students for each political view (GROUP BY multiple columns)

In [ ]:
run_query("""
SELECT Sex, PoliticalViews, COUNT(*) AS cnt
FROM `nyu-datasets.facebook.Profiles`
GROUP BY Sex, PoliticalViews
ORDER BY Sex, cnt DESC
""")

In [ ]:
# Filtering out NULLs first
run_query("""
SELECT Sex, PoliticalViews, COUNT(*) AS cnt
FROM `nyu-datasets.facebook.Profiles`
WHERE Sex IS NOT NULL AND PoliticalViews IS NOT NULL
GROUP BY Sex, PoliticalViews
ORDER BY Sex, cnt DESC
""")

#### Find the most popular TV Shows

In [ ]:
run_query("""
SELECT TVShow, COUNT(*) AS cnt
FROM `nyu-datasets.facebook.FavoriteTVShows`
GROUP BY TVShow
ORDER BY cnt DESC
LIMIT 25
""")

#### Find the most popular Books

In [ ]:
run_query("""
SELECT Book, COUNT(*) AS cnt
FROM `nyu-datasets.facebook.FavoriteBooks`
GROUP BY Book
ORDER BY cnt DESC
LIMIT 25
""")

#### Find the number of students in various relationship statuses

In [ ]:
run_query("""
SELECT Status, COUNT(*) AS cnt
FROM `nyu-datasets.facebook.Relationship`
GROUP BY Status
ORDER BY cnt DESC
""")

#### Find the most popular majors (concentration)

In [ ]:
run_query("""
SELECT Concentration, COUNT(*) AS cnt
FROM `nyu-datasets.facebook.Concentration`
GROUP BY Concentration
ORDER BY cnt DESC
""")

#### List the number of students per birth year

Use `EXTRACT(YEAR FROM date)` to get the year from a date column in BigQuery.

In [ ]:
run_query("""
SELECT EXTRACT(YEAR FROM Birthday) AS birth_year, COUNT(*) AS cnt
FROM `nyu-datasets.facebook.Profiles`
WHERE Birthday IS NOT NULL
GROUP BY birth_year
ORDER BY cnt DESC
""")

---
## `HAVING`

HAVING filters groups based on aggregate values (like WHERE, but for groups).

```sql
SELECT column, AggFunc(column)
FROM table
GROUP BY column
HAVING AggFunc(column) > value
```

**Note:** In BigQuery, you cannot use column aliases in HAVING. You must repeat the aggregate expression.

#### Find movies with more than 100 actors

In [ ]:
run_query("""
SELECT
    movie_id,
    COUNT(*) AS num_roles,
    COUNT(DISTINCT actor_id) AS num_actors
FROM `nyu-datasets.imdb.roles`
GROUP BY movie_id
HAVING COUNT(DISTINCT actor_id) > 100
ORDER BY num_actors DESC
""")

#### Find first names of actors that appear more than 1000 times

In [ ]:
run_query("""
SELECT first_name, COUNT(*) AS cnt
FROM `nyu-datasets.imdb.actors`
GROUP BY first_name
HAVING COUNT(*) > 1000
ORDER BY cnt DESC
""")

#### Find movies where the same actor plays multiple roles

In [ ]:
run_query("""
SELECT
    movie_id,
    COUNT(*) AS num_roles,
    COUNT(DISTINCT actor_id) AS num_actors
FROM `nyu-datasets.imdb.roles`
GROUP BY movie_id
HAVING COUNT(*) <> COUNT(DISTINCT actor_id)
ORDER BY num_actors DESC
LIMIT 20
""")

#### Find actors who play multiple roles (across different movies)

In [ ]:
run_query("""
SELECT
    actor_id,
    COUNT(*) AS num_roles,
    COUNT(DISTINCT movie_id) AS num_movies
FROM `nyu-datasets.imdb.roles`
GROUP BY actor_id
HAVING COUNT(*) <> COUNT(DISTINCT movie_id)
ORDER BY num_movies DESC
LIMIT 20
""")

#### Data quality: Find duplicate movie-genre entries

In [ ]:
run_query("""
SELECT movie_id, genre, COUNT(*) AS cnt
FROM `nyu-datasets.imdb.movies_genres`
GROUP BY movie_id, genre
HAVING COUNT(*) > 1
ORDER BY cnt DESC
""")

#### List only birth years that have at least 10 students

In [ ]:
run_query("""
SELECT EXTRACT(YEAR FROM Birthday) AS birth_year, COUNT(*) AS cnt
FROM `nyu-datasets.facebook.Profiles`
WHERE Birthday IS NOT NULL
GROUP BY birth_year
HAVING COUNT(*) >= 10
ORDER BY cnt DESC
""")

---
## Comparing `WHERE` and `HAVING`

- **WHERE** filters individual rows *before* grouping
- **HAVING** filters groups *after* aggregation

Think of it as: WHERE is for rows, HAVING is for groups.

In [ ]:
# Without WHERE: counts all movies
run_query("""
SELECT COUNT(*) AS all_movies, COUNT(rating) AS rated_movies
FROM `nyu-datasets.imdb.movies`
""")

In [ ]:
# With WHERE: only counts movies that have ratings
run_query("""
SELECT COUNT(*) AS all_movies, COUNT(rating) AS rated_movies
FROM `nyu-datasets.imdb.movies`
WHERE rating IS NOT NULL
""")

---
## `JOIN` and `GROUP BY` Together

Combining JOINs with GROUP BY allows computing statistics across related tables.

### Facebook: JOIN + GROUP BY

#### Show the number of music likes per ProfileID

In [ ]:
run_query("""
SELECT P.ProfileID, COUNT(M.Music) AS num_music_likes
FROM `nyu-datasets.facebook.Profiles` P
INNER JOIN `nyu-datasets.facebook.FavoriteMusic` M ON P.ProfileID = M.ProfileID
GROUP BY P.ProfileID
ORDER BY num_music_likes DESC
LIMIT 20
""")

#### Show the number of music likes per Political View

In [ ]:
run_query("""
SELECT P.PoliticalViews, COUNT(M.Music) AS num_music_likes
FROM `nyu-datasets.facebook.Profiles` P
INNER JOIN `nyu-datasets.facebook.FavoriteMusic` M ON P.ProfileID = M.ProfileID
WHERE P.PoliticalViews IS NOT NULL
GROUP BY P.PoliticalViews
ORDER BY num_music_likes DESC
""")

#### Show the number of music likes per Sex

In [ ]:
run_query("""
SELECT P.Sex, COUNT(M.Music) AS num_music_likes
FROM `nyu-datasets.facebook.Profiles` P
INNER JOIN `nyu-datasets.facebook.FavoriteMusic` M ON P.ProfileID = M.ProfileID
WHERE P.Sex IS NOT NULL
GROUP BY P.Sex
""")

#### Compare INNER JOIN vs LEFT JOIN for counting music likes

In [ ]:
# LEFT JOIN includes profiles with zero music likes
run_query("""
SELECT P.Sex, COUNT(M.Music) AS num_music_likes
FROM `nyu-datasets.facebook.Profiles` P
LEFT JOIN `nyu-datasets.facebook.FavoriteMusic` M ON P.ProfileID = M.ProfileID
WHERE P.Sex IS NOT NULL
GROUP BY P.Sex
""")

### IMDB: JOIN + GROUP BY

#### For each movie genre, list statistics for movies from year 2000

Include: average, min, max ratings, standard deviation, count of rated and total movies

In [ ]:
run_query("""
SELECT
    G.genre,
    COUNT(*) AS num_movies,
    COUNT(M.rating) AS rated_movies,
    ROUND(AVG(M.rating), 2) AS avg_rating,
    MIN(M.rating) AS min_rating,
    MAX(M.rating) AS max_rating,
    ROUND(STDDEV(M.rating), 2) AS std_rating
FROM `nyu-datasets.imdb.movies` M
INNER JOIN `nyu-datasets.imdb.movies_genres` G ON M.id = G.movie_id
WHERE M.year = 2000
GROUP BY G.genre
ORDER BY avg_rating DESC
""")

#### For each director, compute movie statistics

- Number of rated and total movies
- Average, min, max, and standard deviation of ratings
- Limit to directors with at least 40 movies and at least 30 rated movies

In [ ]:
run_query("""
SELECT
    D.id AS director_id,
    D.first_name,
    D.last_name,
    COUNT(*) AS num_movies,
    COUNT(M.rating) AS rated_movies,
    ROUND(AVG(M.rating), 2) AS avg_rating,
    MIN(M.rating) AS min_rating,
    MAX(M.rating) AS max_rating,
    ROUND(STDDEV(M.rating), 2) AS std_rating
FROM `nyu-datasets.imdb.directors` D
JOIN `nyu-datasets.imdb.movies_directors` MD ON D.id = MD.director_id
JOIN `nyu-datasets.imdb.movies` M ON M.id = MD.movie_id
GROUP BY D.id, D.first_name, D.last_name
HAVING COUNT(*) > 40 AND COUNT(M.rating) > 30
ORDER BY avg_rating DESC
""")

#### What roles have the best movie ratings?

- Exclude movies without ratings
- Only roles appearing in at least 10 distinct movies
- Only roles played by at least 10 distinct actors

In [ ]:
run_query("""
SELECT
    R.role,
    COUNT(*) AS num_roles,
    COUNT(DISTINCT R.movie_id) AS num_movies,
    COUNT(DISTINCT R.actor_id) AS num_actors,
    ROUND(AVG(M.rating), 2) AS avg_rating,
    MIN(M.rating) AS min_rating,
    MAX(M.rating) AS max_rating,
    ROUND(STDDEV(M.rating), 2) AS std_rating
FROM `nyu-datasets.imdb.roles` R
JOIN `nyu-datasets.imdb.movies` M ON M.id = R.movie_id
WHERE M.rating IS NOT NULL
GROUP BY R.role
HAVING COUNT(DISTINCT R.movie_id) >= 10 AND COUNT(DISTINCT R.actor_id) >= 10
ORDER BY avg_rating DESC
LIMIT 50
""")

---
## Self-Joins + GROUP BY

#### Find the most common double majors

In [ ]:
run_query("""
SELECT
    C1.Concentration AS major1,
    C2.Concentration AS major2,
    COUNT(*) AS num_students
FROM `nyu-datasets.facebook.Concentration` C1
JOIN `nyu-datasets.facebook.Concentration` C2
    ON C1.ProfileID = C2.ProfileID AND C1.Concentration < C2.Concentration
GROUP BY C1.Concentration, C2.Concentration
ORDER BY num_students DESC
LIMIT 20
""")

#### What are the favorite bands for students that like Radiohead?

In [ ]:
run_query("""
SELECT M2.Music AS also_likes, COUNT(*) AS num_students
FROM `nyu-datasets.facebook.FavoriteMusic` M1
JOIN `nyu-datasets.facebook.FavoriteMusic` M2
    ON M1.ProfileID = M2.ProfileID AND M1.Music <> M2.Music
WHERE M1.Music = 'Radiohead'
GROUP BY M2.Music
ORDER BY num_students DESC
LIMIT 20
""")